# Creación de redes neuronales predictoras de fases

In [8]:
#Lectura de la base de datos
import pandas as pd

#Preprocesamiento de los datos
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

#Métricas de evaluación
from keras import backend as K

#Construcción de la red neuronal
from keras.models import Sequential
from keras import layers

#Construcción de la matriz de incrustación
import numpy as np

In [9]:
#Métricas
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

#Creación de la matriz de incrustación
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [10]:
#Lectura del fichero de datos CSV. Separación por ; e ignorando el espacio inicial
data = pd.read_csv('database_speech07.csv', sep = ';', skipinitialspace = True)

#Parámetros de interés
embedding_dim = 300 #Tamaño del vocabulario de incrustación
vocab_size = 1860 #Tamaño del vocabulario de la red neuronal
maxlen = 200 #Máxima longitud del padding

#Preprocesado de datos
X_data, y_data = data['Transcription'],data['Phase']
#Tokenización
tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(X_data)
X = tokenizer.texts_to_sequences(X_data)
#Padding
X_pad = pad_sequences(X, padding='post', maxlen=maxlen)
#Balanceo
#ROS
ros = RandomOverSampler()
#SMOTE
smote = SMOTE()
X_SM, y_SM = smote.fit_resample(X_pad,y_data)
X_RO, y_RO = ros.fit_resample(X_pad,y_data)
#División en conjunto de entrenamiento y conjunto de evaluación
X_trainSM, X_testSM, y_trainSM, y_testSM = train_test_split(X_SM, y_SM)
X_trainRO, X_testRO, y_trainRO, y_testRO = train_test_split(X_RO, y_RO)
#Construcción del vector de fase
y_TrRO = to_categorical(y_trainRO, num_classes=8)
y_TeRO = to_categorical(y_testRO, num_classes=8)
y_TrSM = to_categorical(y_trainSM, num_classes=8)
y_TeSM = to_categorical(y_testSM, num_classes=8)

#Construcción de la matriz de incrustación
embedding_matrix = create_embedding_matrix(
    'glove/glove.840B.300d.txt',
     tokenizer.word_index, embedding_dim)

#Construcción de la red neuronal
model = Sequential()
#Capa de incrustación
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
#Capa de convolución
model.add(layers.Conv1D(256, 7, activation='relu')) #Cambio de filtro por 128 y cambio de núcleo por 5
#Capa de agrupación
model.add(layers.GlobalMaxPool1D())
#Capa densa
model.add(layers.Dense(20, activation='relu'))
#Capa densa
model.add(layers.Dense(8, activation='sigmoid'))
#Complicación
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',f1_m,precision_m, recall_m])
#Resumen de la red
model.summary()

#Ajuste del modelo a los datos
history = model.fit(X_trainRO, y_TrRO,
                    epochs=60, #Cuantas iteraciones
                    verbose=False,
                    batch_size=10) #Tamaño de nodos que se entrenan

#Métricas
#Conjunto de entrenamiento
loss, accuracy,f1_score, precision, recall = model.evaluate(X_trainRO, y_TrRO, verbose=False)
print("Exactitud de entrenamiento: {:.4f}".format(accuracy))
print("F1 de entrenamiento: {:.4f}".format(f1_score))
print("Precisión de entrenamiento: {:.4f}".format(precision))
print("Memoria de entrenamiento: {:.4f}".format(recall))
#Conjunto de evaluación
loss, accuracy,f1_score, precision, recall = model.evaluate(X_testRO, y_TeRO, verbose=False)
print("Exactitud de entrenamiento: {:.4f}".format(accuracy))
print("F1 de entrenamiento: {:.4f}".format(f1_score))
print("Precisión de entrenamiento: {:.4f}".format(precision))
print("Memoria de entrenamiento: {:.4f}".format(recall))

#Almacenamiento del modelo
#import pickle
#filename = 'modelNNK.model'
#pickle.dump(model, open(filename, 'wb'))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 300)          558000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 194, 256)          537856    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                5140      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 168       
Total params: 1,101,164
Trainable params: 543,164
Non-trainable params: 558,000
_________________________________________________________________

Exactitud d

# Creación de la red neuronal extractora de características

In [11]:
#Lectura del fichero de datos CSV. Separación por ; e ignorando el espacio inicial
data = pd.read_csv('database_speech07.csv', sep = ';', skipinitialspace = True)

#Parámetros de interés
embedding_dim = 300 #Tamaño del vocabulario de incrustación
vocab_size = 1860 #Tamaño del vocabulario de la red neuronal
maxlen = 200 #Máxima longitud del padding

#Preprocesado de datos
X_data, y_data = data['Transcription'],data['Phase']
#Tokenización
tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(X_data)
X = tokenizer.texts_to_sequences(X_data)
#Padding
X_pad = pad_sequences(X, padding='post', maxlen=maxlen)
#Balanceo
#ROS
ros = RandomOverSampler()
#SMOTE
smote = SMOTE()
X_SM, y_SM = smote.fit_resample(X_pad,y_data)
X_RO, y_RO = ros.fit_resample(X_pad,y_data)
#División en conjunto de entrenamiento y conjunto de evaluación
X_trainSM, X_testSM, y_trainSM, y_testSM = train_test_split(X_SM, y_SM)
X_trainRO, X_testRO, y_trainRO, y_testRO = train_test_split(X_RO, y_RO)
#Construcción del vector de fase
y_TrRO = to_categorical(y_trainRO, num_classes=8)
y_TeRO = to_categorical(y_testRO, num_classes=8)
y_TrSM = to_categorical(y_trainSM, num_classes=8)
y_TeSM = to_categorical(y_testSM, num_classes=8)


#Construcción de la red neuronal
model = Sequential()
#Capa de incrustación
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           input_length=maxlen, 
                           trainable=False))
#Capa de convolución
model.add(layers.Conv1D(64, 7, activation='relu')) #Cambio de filtro por 128 y cambio de núcleo por 5
#Capa de agrupación
model.add(layers.GlobalMaxPool1D())
#Capa densa
model.add(layers.Dense(20, activation='relu'))
#Capa densa
model.add(layers.Dense(8, activation='sigmoid'))
#Complicación
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',f1_m,precision_m, recall_m])
#Resumen de la red
model.summary()

#Ajuste del modelo a los datos
history = model.fit(X_trainRO, y_TrRO,
                        validation_data=(X_testRO,y_TeRO),
                        epochs=30,
                        verbose=False,
                        batch_size=10)
#Métricas
#Conjunto de entrenamiento
loss, accuracy,f1_score, precision, recall = model.evaluate(X_trainRO, y_TrRO, verbose=False)
print("Exactitud de entrenamiento: {:.4f}".format(accuracy))
print("F1 de entrenamiento: {:.4f}".format(f1_score))
print("Precisión de entrenamiento: {:.4f}".format(precision))
print("Memoria de entrenamiento: {:.4f}".format(recall))
#Conjunto de evaluación
loss, accuracy,f1_score, precision, recall = model.evaluate(X_testRO, y_TeRO, verbose=False)
print("Exactitud de entrenamiento: {:.4f}".format(accuracy))
print("F1 de entrenamiento: {:.4f}".format(f1_score))
print("Precisión de entrenamiento: {:.4f}".format(precision))
print("Memoria de entrenamiento: {:.4f}".format(recall))

#Almacenamiento del modelo
#import pickle
#filename = 'modelNNK.model'
#pickle.dump(model, open(filename, 'wb'))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 300)          558000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 194, 64)           134464    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                1300      
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 168       
Total params: 693,932
Trainable params: 135,932
Non-trainable params: 558,000
_________________________________________________________________
Exactitud de entrenamiento: 0.9976
F1 de entrenamiento: 0.9909
Precisión de entrenamiento: 0.9897
Memoria de